<a href="https://colab.research.google.com/github/rsolovev/DMU_env/blob/main/DMU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import random

In [86]:
class Vehicle:
  def __init__(self, s_number, start_position, velocity, cache_capacity):
    self.s_number = s_number
    self.position = start_position
    self.velocity = velocity
    self.cache_capacity = cache_capacity
    self.neighbours = []
    self.rsu = None
  
  # print vehicle properties at this time
  def print_vehicle_info(vehicle):
    print("Vehicle #{} is currently at {}, moving with velocity {} m/s, neighbours are {}, cache capacity is {}, assigned to RSU {}"
    .format(vehicle.s_number, vehicle.position, vehicle.velocity, [x.s_number for x in vehicle.neighbours], vehicle.cache_capacity, vehicle.rsu.s_number))

In [87]:
class RSU:
  def __init__(self, s_number, position, cache_capacity, backhaul):
    self.s_number = s_number
    self.position = position
    self.cache_capacity = cache_capacity
    self.backhaul = backhaul
  
  # print RSU properties at this time
  def print_rsu_info(rsu, env):
    assigned_vehicles = env.list_assigned_vehicles(rsu)
    print("RSU #{} is at {}, cache capacity is {}, backhaul rate is {}. List of assigned vehicles: {}"
    .format(rsu.s_number, rsu.position, rsu.cache_capacity, rsu.backhaul, assigned_vehicles))

In [88]:
class V2Vlink:
  def __init__(self):
    pass

In [89]:
class V2Ilink:
  def __init__(self):
    pass

In [90]:
class MapUpdate:
  def __init__(self, size):
    self.size = size

In [98]:
class Environment:
  def __init__(self):
    self.time = 0
    self.vehicles = []
    self.rsus = []
  
  # init the env, start the simulation
  def start(self, n_vehicles, n_rsus):
    self.add_n_vehicles(n_vehicles)
    self.add_n_rsus(n_rsus)
    self.step(0)

  
  # add a vehicle to the environment
  def add_new_vehicle(self, s_number, start_position, velocity, cache_capacity):
        self.vehicles.append(Vehicle(s_number=s_number, start_position=start_position, velocity=velocity, cache_capacity=cache_capacity))
  
  # add n vehicles with random properties
  def add_n_vehicles(self, n):
      cache_sizes = [50,100,150,200,250]
      for i in range(1,n+1):
        self.vehicles.append(Vehicle(s_number=i, start_position=random.randint(0, 200), velocity=random.randint(8, 25), cache_capacity=random.choice(cache_sizes)))
  
  # add an RSU to the environment
  def add_new_rsu(self, s_number, position, cache_capacity, backhaul):
        self.rsus.append(RSU(s_number=s_number, position=position, cache_capacity=cache_capacity, backhaul=backhaul))
  
  # add n RSUs with random properties
  def add_n_rsus(self, n):
    cache_sizes = [5000,1000,1500,2000,2500]
    backhaul_rates = [75, 100, 150]
    rsu_location = 150
    for i in range(1, n+1):
      self.rsus.append(RSU(s_number=i, position=rsu_location, cache_capacity=random.choice(cache_sizes), backhaul=random.choice(backhaul_rates)))
      rsu_location += 300

  
  # iterate for every vehicle and check if new neighbours appeared or someone left the V2V range
  def renew_neighbours(self):
    for vehicle in self.vehicles:
      vehicle.neighbours = []
      for veh in self.vehicles:
        if veh.s_number != vehicle.s_number and abs(veh.position - vehicle.position) <= 30:
          vehicle.neighbours.append(veh)

  # iterate for every vehicle to check if it left the RSU coverage radius and assign new one (or assign to the nearest one)
  def renew_assigned_rsus(self):
    for vehicle in self.vehicles:
      if vehicle.rsu == None:
        vehicle.rsu = self.rsus[0]
      for rsu in self.rsus:
        if abs(rsu.position - vehicle.position) < abs(vehicle.rsu.position - vehicle.position):
          vehicle.rsu = rsu

  
  # recalculates all the vehicle's position after given time
  def renew_positions(self, time):
    for vehicle in self.vehicles:
      vehicle.position = vehicle.position + time*vehicle.velocity
  
  # renew vehicles positions, neighbours, assigned RSUs
  def step(self, time):
    self.renew_positions(time)
    self.renew_neighbours()
    self.renew_assigned_rsus()
    self.time += time
  
  # check which vehicles are assigned to particular RSU
  def list_assigned_vehicles(self, rsu):
    assigned_vehicles = []
    for vehicle in self.vehicles:
      if rsu.s_number == vehicle.rsu.s_number:
        assigned_vehicles.append(vehicle.s_number)
    return assigned_vehicles
  
  # print states of all vehicles and RSUs
  def print_env_info(self):
    print("-----------------------")
    print("TIME: {}".format(self.time))
    print()
    for rsu in self.rsus:
      rsu.print_rsu_info(self)
    print()
    for vehicle in self.vehicles:
      vehicle.print_vehicle_info()
    print("----------------------\n")




In [99]:
test_env = Environment()
test_env.start(n_vehicles=5, n_rsus=5)
test_env.print_env_info()

test_env.step(10)
test_env.print_env_info()

test_env.step(20)
test_env.print_env_info()


-----------------------
TIME: 0

RSU #1 is at 150, cache capacity is 1000, backhaul rate is 75. List of assigned vehicles: [1, 2, 3, 4, 5]
RSU #2 is at 450, cache capacity is 1000, backhaul rate is 150. List of assigned vehicles: []
RSU #3 is at 750, cache capacity is 2000, backhaul rate is 150. List of assigned vehicles: []
RSU #4 is at 1050, cache capacity is 1500, backhaul rate is 150. List of assigned vehicles: []
RSU #5 is at 1350, cache capacity is 1000, backhaul rate is 150. List of assigned vehicles: []

Vehicle #1 is currently at 7, moving with velocity 14 m/s, neighbours are [2, 4, 5], cache capacity is 50, assigned to RSU 1
Vehicle #2 is currently at 26, moving with velocity 10 m/s, neighbours are [1, 4, 5], cache capacity is 200, assigned to RSU 1
Vehicle #3 is currently at 171, moving with velocity 14 m/s, neighbours are [], cache capacity is 100, assigned to RSU 1
Vehicle #4 is currently at 27, moving with velocity 23 m/s, neighbours are [1, 2, 5], cache capacity is 100, 